In [1]:
pip install pandas requests


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import requests
import gzip
from io import BytesIO

C:\Users\cmhcm\AppData\Local\Temp\ipykernel_15152\777464514.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# 데이터 다운로드
def download_pageviews(year, month, day, hour):
    url = (
        f"https://dumps.wikimedia.org/other/pageviews/"
        f"{year}/{year}-{month:0>2}/pageviews-{year}{month:0>2}{day:0>2}-{hour:0>2}0000.gz"
    )
    response = requests.get(url)
    return response.content

# 데이터 압축 해제
def extract_gz(data):
    with gzip.open(BytesIO(data), 'rb') as f:
        return f.read().decode('utf-8')

# 데이터 파싱 및 DataFrame 변환
def parse_to_dataframe(data):
    lines = data.split('\n')
    records = [line.split() for line in lines if line]
    # 데이터의 실제 컬럼 수에 맞춰 수정
    df = pd.DataFrame(records)
    return df

# 하루치 데이터 다운로드 및 DataFrame으로 변환
def fetch_daily_data(year, month, day):
    all_data = []
    for hour in range(2):
        gz_data = download_pageviews(year, month, day, hour)
        extracted_data = extract_gz(gz_data)
        df = parse_to_dataframe(extracted_data)
        all_data.append(df)
    
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

# 예시 사용
year, month, day = 2024, 7, 31
daily_df = fetch_daily_data(year, month, day)

In [4]:
daily_df.columns = ['LanguageCode', 'PageTitle', 'PageViews', 'Metadata']

In [5]:
# 결측치 또는 비어있는 값이 있다면 처리 (예: NaN을 0으로 대체)
daily_df['PageViews'] = daily_df['PageViews'].fillna(0)

# object 타입을 int로 변환
daily_df['PageViews'] = daily_df['PageViews'].astype(int)

In [6]:
threshold = 100
df_filtered = daily_df[daily_df['PageViews'] >= threshold]

In [7]:
# 페이지 유형에 따른 필터링 기준 설정
exclude_prefixes = [
    "Main_Page", "Special:", "Talk:", "User:", "User talk:", 
    "Category:", "Portal:", "File:", "Template:", "Help:", "Wikipedia:",
    "MediaWiki:", "Spécial:", "メインページ"
]

# 특정 유형의 페이지 제외
# PageTitle이 제외할 접두사를 가진 경우 제외
df_filtered = df_filtered[~df_filtered['PageTitle'].str.startswith(tuple(exclude_prefixes))]

In [8]:
# 언어 코드별 페이지 수 확인
language_counts = df_filtered['LanguageCode'].value_counts()
# 문서 수가 10 이하인 언어 코드 찾기
languages_to_remove = language_counts[language_counts <= 10].index
# 해당 언어 코드를 가진 행 제거
df_filtered = df_filtered[~df_filtered['LanguageCode'].isin(languages_to_remove)]

In [9]:
# 인덱스를 기본값으로 재설정하고 새로운 인덱스 부여
df_filtered = df_filtered.reset_index(drop=True)

In [10]:
df_filtered.drop(columns=['Metadata'], inplace=True)

In [11]:
language_to_country = {
    'en': 'United States',
    'ar': 'Saudi Arabia',
    'zh': 'China',
    'es': 'Spain',
    'fr': 'France',
    'de': 'Germany',
    'ru': 'Russia',
    'ko': 'Korea',
    'ja': 'Japan',
    'fa': 'Iran',
    'vi': 'Vietnam',
    'he': 'Israel',        # 히브리어
    'id': 'Indonesia',    # 인도네시아어
    'it': 'Italy',        # 이탈리아어
    'pl': 'Poland',       # 폴란드어
    'pt': 'Portugal',     # 포르투갈어
    'tl': 'Philippines',  # 타갈로그어
    'tr': 'Turkey',       # 터키어
    # 다른 언어 코드와 나라를 추가할 수 있습니다
}

In [12]:
# LanguageCode에서 언어 코드를 추출하여 Country 컬럼 생성
df_filtered['Country'] = df_filtered['LanguageCode'].str.split('.').str[0].map(language_to_country)
df_filtered

,LanguageCode,PageTitle,PageViews,Country
0,ar.m,69_(وضعية_جنسية),122,Saudi Arabia
1,ar.m,X_(فيلم_2022),174,Saudi Arabia
2,ar.m,آل_التنين,104,Saudi Arabia
3,ar.m,أسماء_الله_الحسنى,122,Saudi Arabia
4,ar.m,أصالة_كامل,183,Saudi Arabia
...,...,...,...,...
20036,zh.m,马龙_(乒乓球运动员),179,China
20037,zh.m,高禮澤,123,China
20038,zh.m,高虹安,194,China
20039,zh.m,黃道十二宮,107,China


In [17]:
df_filtered['Country'].unique()

array(['Saudi Arabia', 'Germany', 'United States', 'Spain', 'Iran',
       'France', 'Israel', 'Indonesia', 'Italy', 'Japan', 'Korea',
       'Poland', 'Portugal', 'Russia', 'Philippines', 'Turkey', 'Vietnam',
       'China'], dtype=object)